In [ ]:
import os
import re
import ast
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm.notebook import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

import torch.nn as nn
from torch.optim import AdamW
from sklearn.metrics import classification_report

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1


In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 8
BERT_SPECIES = 'microsoft/deberta-base'
INPUT_CATS = ['src','rel','trg','mis']
OUTPUT_CATS = []
EPOCHS = 5
LEARNING_RATE = 1e-5
LOGGING_INTERVAL = 100

In [ ]:
rawdf = {}
for split in ['train','val','test']:
    dpath = f"../../data/complete_{split}.csv"
    rawdf[split] = pd.read_csv(dpath)

In [ ]:
rawdf['train'].mistype.unique()

In [ ]:
mistypes = ['T Sisters', 'Negation', 'Simple', 'Set Based', 'Hypo-Hyper', 'Gradable', 'Converses', 'Mero-Holo', 'Reverses']
mislocations = ['Target-Head', 'Target-Modifier', 'Relation-Modifier', 'Relation-Head', 'Source-Head', 'Source-Modifier']
num_classes_mistypes = 9
num_classes_mislocations = 6
def process_df(df):
    df = df[(df.mistype.notnull()) & (df.mistype!= 'Doubtful type')]
    df = df[(df.mislocation.notnull()) & (df.mislocation!= 'Doubtful location')]
    df['mistype'] = df.mistype.apply(lambda x: mistypes.index(x))
    df['mislocation'] = df.mislocation.apply(lambda x: mislocations.index(x))
    return df

In [ ]:
for i in ['train','val','test']:
    rawdf[i] = process_df(rawdf[i])

In [ ]:
rawdf['train']['mislocation'].unique()

In [ ]:
input_cats = INPUT_CATS
output_cats = OUTPUT_CATS
include_context = True

In [ ]:
cat2col = {'src':'SSpan','rel':'RSpan','trg':'TSpan','mis':'MSpan'}
col2cat = {col:cat for cat,col in cat2col.items()}

proc_data = {}
for split in ['train','val','test']:
    proc_data[split] = []
    print(split)
    for _, example in tqdm(rawdf[split].iterrows(),total=len(rawdf[split])):
        set_complete = True
        example_feats = {
            'text': None,
            'input_cols': {},
            'output_cols': {},
            'mistype': example['mistype'],
            'mislocation': example['mislocation']
        }
#         text = example['data'].strip() if include_context else example['data'].strip().split('\n')[0]
        text = example['claim'] + ' [SEP] '
#         text = example['claim'] + ' <SEP> '

        #         print(text)
        for io_type, cats in {
            'input_cols': input_cats,
            'output_cols': output_cats,
        }.items():
            for cat in cats:
                col = cat2col[cat]
                span = ast.literal_eval(example[col])
                if any([v==None for v in span]):
                    set_complete = False
                else:
                    if cat == 'mis': 
                        span[0] += len(text)
                        span[1] += len(text)
                    example_feats[io_type][col] = span
        text += example['context']
        example_feats['text'] = text
        if set_complete:
            proc_data[split].append(example_feats)

In [ ]:
pprint(proc_data['train'][3]) 

In [ ]:
cat_tag_map = {
    'SSpan': ['<SRC>','</SRC>'],
    'RSpan': ['<REL>','</REL>'],
    'TSpan': ['<TRG>','</TRG>'],
    'MSpan': ['<MIS>','</MIS>'],
}

ext_data = {}
for split in ['train','val','test']:
    ext_data[split] = []
    print(split)
    for example in tqdm(proc_data[split]):
        new_text = ""
        new_text_split = []
        new_col_spans = {}          # character level
        new_col_spans_split = {}    # word level
        ptr = 0
        col_span_list = []
        for io_type in ['input_cols','output_cols']:
            for cat, span in example[io_type].items():
                col_span_list.append([io_type,cat,span])
        for io_type, cat, span in sorted(col_span_list,key=lambda x: x[2]):
            new_text += example['text'][ptr:span[0]]
            new_text_split.extend([v for v in example['text'][ptr:span[0]].strip().split(' ') if v!=''])
            ptr = span[0]
            
            cat_start, cat_start_split = len(new_text), len(new_text_split)
            
            if io_type == 'input_cols':
                tags = cat_tag_map[cat]
                string = tags[0] + ' ' + example['text'][span[0]:span[1]] + ' ' + tags[1]
                new_text += string
                new_text_split.extend(string.strip().split())
            else:
                string = example['text'][span[0]:span[1]]
                new_text += string
                new_text_split.extend([v for v in string.strip().split() if v!=''])
                
            cat_end, cat_end_split = len(new_text), len(new_text_split)
            
            new_col_spans[cat] = [cat_start,cat_end]
            new_col_spans_split[cat] = [cat_start_split, cat_end_split]
            
            ptr = span[1]

        new_text += example['text'][ptr:]
        new_text_split.extend(example['text'][ptr:].strip().split())
        
        wid2col = ['TXT' for _ in range(len(new_text_split))]
        for col, w_span in new_col_spans_split.items():
            for wid in range(*w_span):
                wid2col[wid] = col
        
        ext_data[split].append({
            'text': new_text,
            'text_split': new_text_split,
            'input_cols': {col:new_col_spans[col] for col in example['input_cols'].keys()},
            'output_cols': {col:new_col_spans[col] for col in example['output_cols'].keys()},
            'input_cols_split': {col:new_col_spans_split[col] for col in example['input_cols'].keys()},
            'output_cols_split': {col:new_col_spans_split[col] for col in example['output_cols'].keys()},
            'wid2col': wid2col,
            'mistype': example['mistype'],
            'mislocation': example['mislocation']
        })
#         print(example['text'])
#         print()
#         print(new_text)
#         print()
#         for col, span in new_output_cols.items():
#             print(col)
#             print(new_text[span[0]:span[1]])
        
#         break
#     break

In [ ]:
ext_data['test'][136]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BERT_SPECIES, add_prefix_space=True)
bert_model = AutoModel.from_pretrained(BERT_SPECIES)

In [ ]:
print(tokenizer.all_special_tokens)
custom_spl_tokens = [tag for cat, tags in cat_tag_map.items() for tag in tags]
# custom_spl_tokens = ['<SEP>'] + [tag for cat, tags in cat_tag_map.items() for tag in tags]
print(custom_spl_tokens)
num_added_toks = tokenizer.add_tokens(custom_spl_tokens, special_tokens=True)
print(num_added_toks)
bert_model.resize_token_embeddings(len(tokenizer))
print(tokenizer.all_special_tokens)
tokenizer.save_pretrained("custom_tokenizer")

In [ ]:
enc = {}
for split in ['train','val','test']:
    enc[split] = tokenizer.batch_encode_plus(
    [eg['text_split'] for eg in ext_data[split]],
    max_length=128,
    add_special_tokens=True,
    is_split_into_words=True,
    padding=True,
    truncation=True,
    return_tensors='pt',
    )

In [ ]:
pipe_data = {} # pipeline-ready data
for split in ['train','val','test']:
    pipe_data[split] = []
    print(split)
    for i in tqdm(range(enc[split]['input_ids'].shape[0])):
        input_ids = enc[split]['input_ids'][i]
        attention_mask = enc[split]['attention_mask'][i]
        sample = ext_data[split][i]
        # output_cats = list(sample['output_cols_split'].keys())
        labels1 = sample['mislocation']
        labels2 = sample['mistype']
        pipe_data[split].append({
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels1': labels1,
            'labels2': labels2,
        })

In [ ]:
pipe_data['test'][136]

In [ ]:
class StringControlDataset(Dataset):
    
    def __init__(self,data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        sample = self.data[idx]
        input_ids = sample['input_ids']
        attention_mask = sample['attention_mask']
        zeros = torch.zeros(num_classes_mislocations)
        zeros[sample['labels1']] = 1
        labels1 = zeros
        zeros2 = torch.zeros(num_classes_mistypes)
        zeros2[sample['labels2']] = 1
        labels2 = zeros2
        return input_ids, attention_mask, labels1, labels2

In [ ]:
strctrl_dataset = {}
for split in ['train','val','test']:
    strctrl_dataset[split] = StringControlDataset(pipe_data[split])

In [ ]:
strctrl_dataloader = {}
for split in ['train','val','test']:
    strctrl_dataloader[split] = DataLoader(strctrl_dataset[split],
                                           batch_size=BATCH_SIZE,
#                                            shuffle=True
                                          )

In [ ]:
class StringControlModel(nn.Module):
    def __init__(self,bert_model,out_classes=9):
        super().__init__()
        self.bert_model = bert_model
        self.out_classes = out_classes
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.pre_classifier_1 = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 6)
        self.pre_classifier_2 = torch.nn.Linear(768, 768)
        self.classifier2 = torch.nn.Linear(768, 9)
        
    def forward(self,input_ids,attention_mask):
        emb = self.bert_model(input_ids=input_ids,attention_mask=attention_mask)
        hidden_state_1 = emb[0]
        pooler = hidden_state_1[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        pooler_1 = self.pre_classifier_1(pooler)
        pooler_1 = torch.nn.Tanh()(pooler_1)
        pooler_1 = self.dropout(pooler_1)
        output1 = self.classifier(pooler_1)
        pooler_2 = self.pre_classifier_2(pooler)
        pooler_2 = torch.nn.Tanh()(pooler_2)
        pooler_2 = self.dropout(pooler_2)
        output2 = self.classifier2(pooler_2)
        return output1, output2

In [ ]:
model = StringControlModel(bert_model,out_classes=num_classes_mislocations)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"trainable params: {trainable_params}")
print(f"    total params: {total_params}")
print(f"                : {round(100*trainable_params/total_params,2)}%")

model = model.to(DEVICE)

In [ ]:
epochs = EPOCHS
loss_fnc = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(),lr=LEARNING_RATE)
logging_interval = LOGGING_INTERVAL

In [ ]:
# Training
for e in range(epochs):
    print(f'running epoch {e+1} of {epochs}')
    model.train()
    batch_loss = []
    epoch_loss = []
    for bidx, batch in tqdm(enumerate(strctrl_dataloader['train']),total=len(strctrl_dataloader['train'])):
        
        input_ids, attention_mask, labels1, labels2 = [v.to(DEVICE) for v in batch]
        out1, out2 = model(input_ids, attention_mask)
        # labels = torch.permute(labels,(0,2,1))
        # print(out1.shape, out2.shape, labels1.shape, labels2.shape)
        optimizer.zero_grad()
        loss = loss_fnc(out1,labels1) + loss_fnc(out2,labels2)
        loss.backward()
        optimizer.step()
        
        epoch_loss.append(loss.item())
        batch_loss.append(loss.item())
        if len(batch_loss) > logging_interval:
            batch_loss.pop(0)
        if (bidx+1) % logging_interval == 0:
            print(f'train loss: {np.mean(batch_loss)}')
    print(f"epoch loss: {np.mean(epoch_loss)}")
    fin_targets1=[]
    fin_outputs1=[]
    fin_targets2=[]
    fin_outputs2=[]
    batch_loss = []
    epoch_loss = []
    with torch.no_grad():
        for bidx, batch in tqdm(enumerate(strctrl_dataloader['val']),total=len(strctrl_dataloader['val'])):
            input_ids, attention_mask, labels1, labels2 = [v.to(DEVICE) for v in batch]
            out1, out2 = model(input_ids, attention_mask)
            # labels = torch.permute(labels,(0,2,1))
            # print(out1.shape, out2.shape, labels1.shape, labels2.shape)
            optimizer.zero_grad()
            loss = loss_fnc(out1,labels1) + loss_fnc(out2,labels2)
            fin_targets1.extend(labels1.cpu().detach().numpy().tolist())
            fin_outputs1.extend(out1.cpu().detach().numpy().tolist())
            fin_targets2.extend(labels2.cpu().detach().numpy().tolist())
            fin_outputs2.extend(out2.cpu().detach().numpy().tolist())
    fin_targets1 = [fin_targets1[i].index(max(fin_targets1[i])) for i in range(len(fin_targets1))]
    fin_outputs1 = [fin_outputs1[i].index(max(fin_outputs1[i])) for i in range(len(fin_outputs1))]
    fin_targets2 = [fin_targets2[i].index(max(fin_targets2[i])) for i in range(len(fin_targets2))]
    fin_outputs2 = [fin_outputs2[i].index(max(fin_outputs2[i])) for i in range(len(fin_outputs2))]
    print(classification_report(fin_targets1, fin_outputs1))
    print(classification_report(fin_targets2, fin_outputs2))
#         break
#     break

In [ ]:
fin_targets1=[]
fin_outputs1=[]
fin_targets2=[]
fin_outputs2=[]
batch_loss = []
epoch_loss = []
with torch.no_grad():
    for bidx, batch in tqdm(enumerate(strctrl_dataloader['val']),total=len(strctrl_dataloader['val'])):
        input_ids, attention_mask, labels1, labels2 = [v.to(DEVICE) for v in batch]
        out1, out2 = model(input_ids, attention_mask)
        # labels = torch.permute(labels,(0,2,1))
        # print(out1.shape, out2.shape, labels1.shape, labels2.shape)
        optimizer.zero_grad()
        loss = loss_fnc(out1,labels1) + loss_fnc(out2,labels2)
        fin_targets1.extend(labels1.cpu().detach().numpy().tolist())
        fin_outputs1.extend(out1.cpu().detach().numpy().tolist())
        fin_targets2.extend(labels2.cpu().detach().numpy().tolist())
        fin_outputs2.extend(out2.cpu().detach().numpy().tolist())
fin_targets1 = [fin_targets1[i].index(max(fin_targets1[i])) for i in range(len(fin_targets1))]
fin_outputs1 = [fin_outputs1[i].index(max(fin_outputs1[i])) for i in range(len(fin_outputs1))]
fin_targets2 = [fin_targets2[i].index(max(fin_targets2[i])) for i in range(len(fin_targets2))]
fin_outputs2 = [fin_outputs2[i].index(max(fin_outputs2[i])) for i in range(len(fin_outputs2))]
print(classification_report(fin_targets1, fin_outputs1))
print(classification_report(fin_targets2, fin_outputs2))
